# WEBTOON ORIGINALS

This notebook serves to `GET` data from the Webtoon API on RapidAPI and convert the `JSON` repsonse to `CSV` format.

[Webtoon API](https://rapidapi.com/apidojo/api/webtoon/)

## Requirements

**Libraries**

- requests

**API Key**

A RapidAPI key will be required to get data.

If you do not have a RapidAPI account, you can get a key at [Webtoon API on RapidAPI](https://rapidapi.com/apidojo/api/webtoon/) by going to "Pricing", subscribing to the Basic plan, and creating an account.

If you already have an account, subscribe to the API in order to use it.

In [ ]:
from requests import request
from csv import writer

Function that makes the API call. Returns a JSON object.

In [ ]:
def api_call(lang):
   rapid_api_key = "" # rapid api key here
   
   url = "https://webtoon.p.rapidapi.com/originals/titles/list"
   
   querystring = {"language": lang}
   
   headers = {
      "x-rapidapi-host": "webtoon.p.rapidapi.com",
      "x-rapidapi-key": rapid_api_key
   }
   
   response = request("GET", url, headers = headers, params = querystring)
   
   titles = response.json()["message"]["result"]["titleList"]["titles"]
   
   return(titles)

Function that makes the csv. Does a bit of cleaning and changes the header to be more readable.

In [ ]:
def make_data(titles, lang):
   dataset = open("webtoon_originals_" + lang + ".csv", "w", newline = "", encoding = "utf-8")
   
   csv_writer = writer(dataset, delimiter = ",")
   
   title_keys = ["titleNo", "title", "representGenre", "titleAuthorList", "weekday",
                 "totalServiceEpisodeCount", "favoriteCount", "starScoreAverage", "readCount",
                 "likeitCount", "restTerminationStatus", "dailyPassTitle", "synopsis"
                ]
   header = ["title_id", "title", "genre", "authors", "weekdays", "length", "subscribers",
             "rating", "views", "likes", "status", "daily_pass", "synopsis"
            ]
   
   csv_writer.writerow(header)
   
   for title in titles:
      title_vals = []
      for k in title_keys:
         auth_list = []
         wkdy_list = []
         if k == "titleAuthorList":
            for auth in title[k]:
               auth_list.append(auth["authorName"])
            auth_str = ",".join(auth_list)
            title_vals.append(auth_str)
         elif k == "weekday":
            for wkdy in title[k]:
               wkdy_list.append(wkdy)
            wkdy_str = ",".join(wkdy_list)
            title_vals.append(wkdy_str)
         elif k == "restTerminationStatus":
            if title[k] == "SERIES":
               title_vals.append("ONGOING")
            elif title[k] == "TERMINATION":
               title_vals.append("COMPLETED")
            elif title[k] == "REST":
               title_vals.append("HIATUS")
            else:
               title_vals.append(title[k])
         else:
            title_vals.append(title[k])
      csv_writer.writerow(title_vals)
   
   dataset.close()

Iterates though all the languages and makes the API call and csv for each language.

In [ ]:
langs = ["en", "zh-hant", "th", "id", "es", "fr", "de"]

for lang in langs:
   titles = api_call(lang)
   make_data(titles, lang)